# Code Division Multiplexing: HTC Tutorial using Jupyter notebooks

## Student project in High Throughput Computing

Author: Richard T. Jones, University of Connecticut, richard.t.jones(at)uconn.edu
<br>Created: March 19, 2022


Define a class for CDM encoding and decoding.

In [1]:
import numpy as np

class CDMcode:
    def __init__(self, log2length):
        """
        Constructs a new CDMcode object of length 2**log2length.
        Initialize the first code to a random bit string, and the
        subsequent log2length codes to the complete set of codes
        that are orthogonal to the first.
        """
        self.codelen = 1 << log2length
        arr = np.random.randint(2, size=self.codelen)
        code0 = np.array(arr, dtype=int)
        count = np.arange(self.codelen, dtype=int)
        self.code = [code0]
        self.coda = [2 * code0 - 1]
        self.codi = [-2 * code0 + 1]
        self.coda8 = [np.array(self.coda[0], dtype=np.byte)]
        self.codi8 = [np.array(self.codi[0], dtype=np.byte)]
        for n in range(log2length):
            coden = np.bitwise_xor(code0, (count >> n) & 1)
            self.code.append(coden)
            self.coda.append(2 * coden - 1)
            self.codi.append(-2 * coden + 1)
            self.coda8.append(np.array(self.coda[-1], dtype=np.byte))
            self.codi8.append(np.array(self.codi[-1], dtype=np.byte))
        self.codasum = [np.sum(coda) for coda in self.coda]
    
    def encode(self, message, chan):
        """
        Takes in a message bytestring and expands it into a signal
        array encoded using the code for channel chan, returning it
        as a numpy array of byte containing the bit values of the
        encoded message, length = len(message) * codelen * 8. The
        bit order in the encoded signal is big-endian.
        """
        signal = np.array([], dtype=np.byte)
        for mbyte in message:
            for mbit in reversed(range(8)):
                if (mbyte >> mbit) % 2 == 0:
                    signal = np.concatenate((signal, self.codi8[chan]))
                else:
                    signal = np.concatenate((signal, self.coda8[chan]))
        return signal
    
    def decode(self, signal, chan):
        """
        Takes in a signal array of bytes and decode any messages found
        in code channel chan. Null bits in the signal are ignored. Valid
        message bits are expected to occur in sequences of 8 bits,
        otherwise a warning message is printed and the method returns
        without any further processing of signal.
        """
        dotmean = np.sum(signal) * self.codasum[chan] / len(signal)
        pthresh = dotmean + self.codelen / 2
        nthresh = dotmean - self.codelen / 2
        message = bytearray()
        mbyte = 0
        mbit = 7
        pos = 0
        while pos + self.codelen <= len(signal):
            sdotc = self.dot(signal, chan, pos)
            #print("sdotc at pos={0} is {1}, pthresh={2}, nthresh={3}".format(pos, sdotc, pthresh, nthresh))
            if sdotc > pthresh:
                mbyte += (1 << mbit)
            elif sdotc > nthresh:
                if mbit != 7:
                    print("CDMcode.decode warning:",
                          "incomplete character found at position",
                          pos / self.codelen)
                return bytes(message)
            pos += self.codelen
            mbit -= 1
            if mbit < 0:
                message.append(mbyte)
                mbyte = 0
                mbit = 7
        return bytes(message)
    
    def dot(self, signal, chan, pos=0):
        """
        Take the inner product between the array signal[pos,pos+codelen]
        and the code for channel chan. The signal array is assumed to
        be at least of length pos + codelen.
        """
        return np.dot(self.coda[chan], signal[pos:pos+self.codelen])

ModuleNotFoundError: No module named 'numpy'

## Othogonality check

Check orthogonality of the codes in one or two special cases, N < 10. This "inner product" is defined as the sum over +1 for each bit position with the same value in the two vectors, and -1 for each bit position with the opposite value. You should verify that the inner product of any code with itself equals the number of bits, whereas the inner product is zero when taken with any of the other codes in the set.

In [ ]:
N = 3
c = CDMcode(N)
for n in range(N+1):
    print("c[{0}]".format(n), ": ", end='')
    print(c.code[n])
    for m in range(n+1):
        print("   c[{0}] . c[{1}] = {2}".format(n, m, c.dot(c.encode(b'\x80', n), m)))


## Encode a message
Create a noisy baseline signal and encode a single message into it. Plot the signal, its amplitude distribution and its autocorrelation.

In [ ]:
N = 24
chan = 3
c = CDMcode(N)
message = b"around"# and around the rugged rocks the ragged rascal ran"
signal = np.array([], dtype=np.byte)
charlen = 8 * c.codelen
for a in message:
    arr = np.array(np.random.normal(64, 12.5, charlen), dtype=np.byte)
    signal = np.concatenate((signal, arr))
signal += c.encode(np.frombuffer(message, dtype=np.byte), chan)

message2 = b"but"# not without his umbrella!"
message2 = message2.ljust(len(message), b'\x00')
signal += c.encode(np.frombuffer(message2, dtype=np.byte), chan + 10)

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
p1 = axes[0].plot(signal[:100])
p2 = axes[1].hist(signal, 128, (0, 128))

## Decode a signal, look for messages
Search the encoded signal for messages on any channel.

In [ ]:
print("signal has length", len(signal), "bytes")
for chan in range(N + 1):
    msg = c.decode(signal, chan)
    if len(msg) > 0:
        print('message found on channel {0}: "{1}"'.format(chan, msg.decode()))